In [20]:
import torch

def ruiz_precondition(c, K, q, l, u, device='cpu', max_iter=20, eps=1e-6):
    """
    Performs Ruiz equilibration (scaling) on the standard-form linear program using GPU tensors.

    This is done to improve the numerical stability of iterative solvers, especially for
    ill-conditioned problems.

    Standard form of the LP:
        minimize     cᵀx
        subject to   Gx ≥ h
                     Ax = b
                     l ≤ x ≤ u

    Inputs:
    -------
    c  : (n x 1) torch tensor — objective function vector
    K  : ((m_ineq + m_eq) x n) torch tensor — constraint matrix (stacked G and A)
    q  : ((m_ineq + m_eq) x 1) torch tensor — RHS vector (stacked h and b)
    l  : (n x 1) torch tensor — lower bounds on variables
    u  : (n x 1) torch tensor — upper bounds on variables
    max_iter : int — number of scaling iterations to perform (default: 20)

    Outputs:
    --------
    K_s : ((m_ineq + m_eq) x n) torch tensor — scaled constraint matrix (stacked G and A)
    c_s : (n x 1) torch tensor — scaled objective vector
    q_s : ((m_ineq + m_eq) x 1) torch tensor — scaled RHS vector (stacked h and b)
    l_s : (n x 1) torch tensor — scaled lower bounds
    u_s : (n x 1) torch tensor — scaled upper bounds
    D_col : (n x 1) torch tensor — final column scaling factors (for rescaling solution)
    m_ineq : int — number of inequality constraints (used for slicing G vs A in K_s if needed)

    Notes:
    ------
    - The scaling preserves feasibility and optimality but improves numerical conditioning.
    - You must rescale your solution after solving using D_col (and D_row if needed).
    """

    # --- Scaling Loop ---
    K_s, c_s, q_s, l_s, u_s = K.clone(), c.clone(), q.clone(), l.clone(), u.clone()
    m, n = K_s.shape

    D_row = torch.ones((m, 1), dtype=K.dtype, device=device)
    D_col = torch.ones((n, 1), dtype=K.dtype, device=device)

    for i in range(max_iter):
        row_norms = torch.sqrt(torch.linalg.norm(K_s, ord=torch.inf, dim=1, keepdim=True))
        row_norms[row_norms < eps] = 1.0
        D_row /= row_norms
        K_s /= row_norms

        col_norms = torch.sqrt(torch.linalg.norm(K_s, ord=torch.inf, dim=0, keepdim=True))
        col_norms[col_norms < eps] = 1.0
        D_col /= col_norms.T
        K_s /= col_norms

        if (torch.max(torch.abs(1 - row_norms)) < eps and
            torch.max(torch.abs(1 - row_norms)) < eps):
            break

    c_s *= D_col
    q_s *= D_row
    l_s /= D_col
    u_s /= D_col

    return K_s, c_s, q_s, l_s, u_s, (D_col, D_row, K, c, q, l, u)

def primal_weight_update(x_prev, x, y_prev, y, omega, smooth_theta):
    diff_y_norm = torch.linalg.norm(y_prev - y, 2)
    diff_x_norm = torch.linalg.norm(x_prev - x, 2)
    if diff_x_norm > 0 and diff_y_norm > 0:
        omega = torch.exp(smooth_theta * (torch.log(diff_y_norm/diff_x_norm)) + (1-smooth_theta)*torch.log(omega))
    return omega

In [21]:
import torch

def project_lambda_box(grad, is_neg_inf, is_pos_inf):
    """
    Projects the gradient onto the normal cone of the feasible region defined by bounds l and u.

    For each i:
      - If l[i] == -inf and u[i] == +inf: projection is 0
      - If l[i] == -inf and u[i] is real: clamp to ≤ 0 (R⁻)
      - If l[i] is real and u[i] == +inf: clamp to ≥ 0 (R⁺)
      - If both are finite: no projection (keep full value)

    Args:
        grad: (n, 1) gradient vector (torch tensor)
        l: (n, 1) lower bounds (torch tensor)
        u: (n, 1) upper bounds (torch tensor)

    Returns:
        projected: (n, 1) projected gradient (interpreted as λ)
    """
    projected = torch.zeros_like(grad)

    # Case 1: (-inf, +inf) → {0}
    unconstrained = is_neg_inf & is_pos_inf
    projected[unconstrained] = 0.0

    # Case 2: (-inf, real) → R⁻ → clamp at 0 from above
    neg_only = is_neg_inf & ~is_pos_inf
    projected[neg_only] = torch.clamp(grad[neg_only], max=0.0)

    # Case 3: (real, +inf) → R⁺ → clamp at 0 from below
    pos_only = ~is_neg_inf & is_pos_inf
    projected[pos_only] = torch.clamp(grad[pos_only], min=0.0)

    # Case 4: (real, real) → full space → keep gradient
    fully_bounded = ~is_neg_inf & ~is_pos_inf
    projected[fully_bounded] = grad[fully_bounded]

    return projected

def spectral_norm_estimate_torch(K, num_iters=10):
  """
  Estimates the spectral norm of a matrix K with enough acuracy to use in
  setting the step size of the PDHG algorithm.
  """

  b = torch.randn(K.shape[1], 1, device=K.device)
  for _ in range(num_iters):
      b = K.T @ (K @ b)
      b /= torch.norm(b)
  return torch.norm(K @ b)

def compute_residuals_and_duality_gap(x, y, c, q, K, m_ineq, is_neg_inf, is_pos_inf, l_dual, u_dual):
    """
    Computes the primal and dual residuals, duality gap, and KKT error.

    Args:
        x (torch.Tensor): Primal variable.
        y (torch.Tensor): Dual variable.
        c (torch.Tensor): Coefficients for the primal objective.
        q (torch.Tensor): Right-hand side vector for the constraints.
        K (torch.Tensor): Constraint matrix.
        m_ineq (int): Number of inequality constraints.
        omega (float): Scaling factor for the dual update.
        is_neg_inf (torch.Tensor): Boolean mask for negative infinity lower bounds.
        is_pos_inf (torch.Tensor): Boolean mask for positive infinity upper bounds.
        l_dual (torch.Tensor): Lower bounds for the dual variables.
        u_dual (torch.Tensor): Upper bounds for the dual variables.
    Returns:
        primal_residual (torch.Tensor): Norm of the primal residual.
        dual_residual (torch.Tensor): Norm of the dual residual.
        duality_gap (torch.Tensor): Duality gap.
    """
    # Primal and dual objective
    grad = c - K.T @ y
    prim_obj = (c.T @ x).flatten()
    dual_obj = (q.T @ y).flatten()

    # Lagrange multipliers from box projection
    lam = project_lambda_box(grad, is_neg_inf, is_pos_inf)
    lam_pos = (l_dual.T @ torch.clamp(lam, min=0.0)).flatten()
    lam_neg = (u_dual.T @ torch.clamp(lam, max=0.0)).flatten()

    adjusted_dual = dual_obj + lam_pos + lam_neg
    duality_gap = adjusted_dual - prim_obj

    # Primal residual (feasibility)
    full_residual = K @ x - q
    residual_ineq = torch.clamp(full_residual[:m_ineq], max=0.0)
    residual_eq = full_residual[m_ineq:]
    primal_residual = torch.norm(torch.vstack([residual_eq, residual_ineq]), p=2).flatten()

    # Dual residual (change in x)
    dual_residual = torch.norm(grad - lam, p=2).flatten()

    return primal_residual, dual_residual, duality_gap, prim_obj, adjusted_dual

def KKT_error(x, y, c, q, K, m_ineq, omega, is_neg_inf, is_pos_inf, l_dual, u_dual, device):
      """
      Computes the KKT error using global variables.
      """
      omega_sqrd = omega ** 2
      # Compute primal and dual residuals, and duality gap
      primal_residual, dual_residual, duality_gap, _ , _ = compute_residuals_and_duality_gap(x, y, c, q, K, m_ineq, is_neg_inf, is_pos_inf, l_dual, u_dual)
      # Compute the error
      KKT = torch.sqrt(omega_sqrd * primal_residual ** 2 + (dual_residual ** 2) / omega_sqrd + duality_gap ** 2)

      return KKT

def check_termination(primal_residual, dual_residual, duality_gap, prim_obj, adjusted_dual, q_norm, c_norm, tol):
    """
    Checks the termination conditions for the PDHG algorithm.
    Args:
        primal_residual (torch.Tensor): Norm of the primal residual.
        dual_residual (torch.Tensor): Norm of the dual residual.
        duality_gap (torch.Tensor): Duality gap.
        prim_obj (torch.Tensor): Primal objective value.
        adjusted_dual (torch.Tensor): Adjusted dual objective value.
        q_norm (float): Norm of the right-hand side vector q.
        c_norm (float): Norm of the coefficients vector c.
        tol (float): Tolerance for stopping criterion.
    Returns:
        bool: True if termination conditions are met, False otherwise.
    """
    cond1 = primal_residual <= tol * (1 + q_norm)
    cond2 = dual_residual <= tol * (1 + c_norm)
    cond3 = duality_gap <= tol * (1 + abs(prim_obj) + abs(adjusted_dual))
    return cond1 and cond2 and cond3

In [ ]:
import argparse
import torch
import os

# Instead of this
# args = parse_args()

# Do this manually
class Args:
    device = 'gpu'
    instance_path = 'PDLP-AMD-RIPS/datasets/Netlib/infeasible'
    tolerance = 1e-8
    output_path = '/content/PDLP-AMD-RIPS/results'
    precondition = True
    primal_weight_update = False
    adaptive_stepsize = False
    verbose = False
    support_sparse = False

args = Args()

def parse_args():
    parser = argparse.ArgumentParser(description='Run LP solver with configuration options.')

    parser.add_argument('--device', type=str, choices=['cpu', 'gpu', 'auto'], default='auto',
                        help="Device to run on: 'cpu', 'gpu', or 'auto' (default: auto)")
    parser.add_argument('--instance_path', type=str, default='feasible',
                        help="Path to folder containing MPS instances (default: 'feasible')")
    parser.add_argument('--tolerance', type=float, default=1e-2,
                        help="Tolerance for stopping criterion (default: 1e-2)")
    parser.add_argument('--output_path', type=str, default='output',
                        help="Directory where outputs will be saved (default: 'output')")
    parser.add_argument('--precondition', action='store_true',
                        help="Enable Ruiz preconditioning (default: False)")
    parser.add_argument('--primal_weight_update', action='store_true',
                        help="Enable primal weight update (default: False)")
    parser.add_argument('--adaptive_stepsize', action='store_true',
                        help="Enable adaptive stepsize for PDLP (default: False)")
    parser.add_argument('--verbose', action='store_true',
                        help="Enable verbose output (default: False)")
    parser.add_argument('--support_sparse', action='store_true',
                        help="Support sparse matrices operations(default: False)")
    return parser.parse_args()

if __name__ == '__main__':
    args = Args #parse_args()

    # --- Device Selection ---
    if args.device == 'auto' or args.device == 'gpu':
        if torch.cuda.is_available():
            device = torch.device('cuda')
            print(f"PyTorch is using ROCm/CUDA device: {torch.cuda.get_device_name(0)}")
        else:
            device = torch.device('cpu')
            print("ROCm/CUDA not available. PyTorch is using CPU.")
    else:
        device = torch.device(args.device)
        print(f"PyTorch is using device: {device}")

    # --- Configuration ---
    mps_folder_path = args.instance_path
    tol = args.tolerance
    output_path = args.output_path
    precondition = args.precondition
    primal_weight_update = args.primal_weight_update
    adaptive_stepsize = args.adaptive_stepsize
    verbose=args.verbose
    support_sparse = args.support_sparse
    results = []

    # --- Get all MPS files from the folder ---
    mps_files = sorted([f for f in os.listdir(mps_folder_path) if f.endswith('.mps')])

    for mps_file in mps_files:
        mps_file_path = os.path.join(mps_folder_path, mps_file)
        print(f"\nProcessing {mps_file_path}...")
        try:
            # --- Load problem ---
            c, K, q, m_ineq, l, u= mps_to_standard_form(mps_file_path, device=device, support_sparse=support_sparse, verbose=verbose)
        except Exception as e:
            print(f"Failed to load MPS file: {mps_file_path}. Error: {e}")
            results.append({
                'File': mps_file,
                'Objective': 'N/A',
                'Iterations (k)': 'N/A',
                'Time (s)': 'N/A',
                'Status': f'Failed to load: {e}'
            })
            continue

        try:
            with Timer("Solve time") as t:
                # PRECONDITION
                if precondition:
                    K, c, q, l, u, dt_precond = ruiz_precondition(c, K, q, l, u, device = device)

                x, prim_obj, k, n, j = pdlp_algorithm(K, m_ineq, c, q, l, u, device, max_iter=10000000, tol=tol, verbose=True, restart_period=40, precondition=precondition,primal_update=primal_weight_update, adaptive=adaptive_stepsize, data_precond=dt_precond)

                print(f"Objective value: {prim_obj:.4f}")

            time_elapsed = t.elapsed
            print(f"Took {time_elapsed:.4f} seconds.")
        except Exception as e:
            print(f"Solver failed for {mps_file}. Error: {e}")

    # Create output directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # --- Call your solver/main logic here ---
    print(f"Instance path: {mps_folder_path}")
    print(f"Tolerance: {tol}")
    print(f"Output path: {output_path}")

PyTorch is using ROCm/CUDA device: AMD Instinct MI210

Processing PDLP-AMD-RIPS/datasets/Netlib/infeasible/bgdbg1.mps...
Artificial restart at iteration 40 using the Current iterate.
[40] Primal Obj: 0.0000, Adjusted Dual Obj: 116.6342, Gap: 9.91e-01, Prim Res: 8.06e-02, Dual Res: 1.02e-02

Artificial restart at iteration 40 using the Average iterate.
[80] Primal Obj: 0.0000, Adjusted Dual Obj: 454.1176, Gap: 9.98e-01, Prim Res: 7.12e-02, Dual Res: 1.33e-02

Artificial restart at iteration 80 using the Average iterate.
[160] Primal Obj: 0.0000, Adjusted Dual Obj: 716.5501, Gap: 9.99e-01, Prim Res: 7.01e-02, Dual Res: 1.77e-03

Artificial restart at iteration 120 using the Average iterate.
[280] Primal Obj: 0.0000, Adjusted Dual Obj: 1134.0616, Gap: 9.99e-01, Prim Res: 7.00e-02, Dual Res: 2.89e-04

Artificial restart at iteration 160 using the Average iterate.
[440] Primal Obj: 0.0881, Adjusted Dual Obj: 1695.9414, Gap: 9.99e-01, Prim Res: 7.00e-02, Dual Res: 2.92e-04

Artificial restar

In [1]:
!cd PDLP-AMD-RIPS
!rm -rf PDLP-AMD-RIPS
!git clone --branch main https://github.com/SimplySnap/PDLP-AMD-RIPS.git

Cloning into 'PDLP-AMD-RIPS'...
remote: Enumerating objects: 1730, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 1730 (delta 61), reused 36 (delta 21), pack-reused 1631 (from 2)
Receiving objects: 100% (1730/1730), 343.40 MiB | 24.71 MiB/s, done.
Resolving deltas: 100% (910/910), done.
Updating files: 100% (427/427), done.


In [31]:
import torch

def pdlp_algorithm(K, m_ineq, c, q, l, u, device, max_iter=100_000, tol=1e-4, verbose=True, restart_period=40, precondition=False, primal_update=False, adaptive=False, data_precond=None):
    '''
    Main PDLP algorithm implementation with integrated infeasibility detection.
    Args:
        K (torch.Tensor): Constraint matrix.
        m_ineq (int): Number of inequality constraints.
        c (torch.Tensor): Coefficients for the primal objective.
        q (torch.Tensor): Right-hand side vector for the constraints.
        l (torch.Tensor): Lower bounds for the primal variable.
        u (torch.Tensor): Upper bounds for the primal variable.
        device (torch.device): Device to run the algorithm on (CPU or GPU).
        max_iter (int): Maximum number of iterations.
        tol (float): Tolerance for convergence.
        verbose (bool): Whether to print detailed output.
        restart_period (int): Number of iterations between restarts.
        precondition (bool): Whether to use preconditioning.
        primal_update (bool): Whether to perform primal weight updates.
        adaptive (bool): Whether to use adaptive stepsize.
        data_precond (tuple): Preconditioned data (D_col, D_row, K_unscaled, c_unscaled, q_unscaled, l_unscaled, u_unscaled) if precondition is True.

    Returns:
        x (torch.Tensor): Optimal primal variable.
        prim_obj (float): Optimal primal objective value.
        k (int): Total number of iterations.
        n (int): Number of restart loops.
        j (int): KKT pass counter.
    '''
    # Recover G, A, h, b for infeasibility detection
    m_eq = K.shape[0] - m_ineq
    if m_ineq > 0:
        G_mat = K[:m_ineq]
        h = q[:m_ineq]
    else:
        G_mat = torch.zeros((0, K.shape[1]), device=device)
        h = torch.zeros((0, 1), device=device)
    if m_eq > 0:
        A_mat = K[m_ineq:]
        b = q[m_ineq:]
    else:
        A_mat = torch.zeros((0, K.shape[1]), device=device)
        b = torch.zeros((0, 1), device=device)

    # Dual bound masks
    is_neg_inf = torch.isinf(l) & (l < 0)
    is_pos_inf = torch.isinf(u) & (u > 0)

    l_dual = l.clone()
    u_dual = u.clone()
    l_dual[is_neg_inf] = 0
    u_dual[is_pos_inf] = 0

    q_norm = torch.linalg.norm(q, 2)
    c_norm = torch.linalg.norm(c, 2)

    # Initial step-size
    eta = 0.9 / spectral_norm_estimate_torch(K, num_iters=100)
    omega = c_norm / q_norm if q_norm > 1e-6 and c_norm > 1e-6 else torch.tensor(1.0)

    theta = 1.0

    # Restart Parameters [Sufficient, Necessary, Artificial]
    beta = [0.2, 0.8, 0.36]

    # Initialize primal and dual
    x = torch.zeros((c.shape[0], 1), device=device)
    y = torch.zeros((K.shape[0], 1), device=device)

    # Infeasibility detection parameters
    tol_eq = 1e-2
    tol_feas = 1e-2
    tol_obj = 1e-2
    tol_dual_eq = 1e-2
    tol_cert = 1e-2

    # Trackers for infeasibility detection
    x_prev_iter = x.clone()
    y_prev_iter = y.clone()
    lam_prev = torch.zeros_like(x)
    certificate_flag = None
    certificate_iter = None

    # Counters
    n = 0 # Outer Loop Counter
    k = 0 # Total Iteration Counter
    j = 0 # Kkt pass Counter

    # Initialize Previous KKT Error
    KKT_first = 0 # The actual KKT error of the very first point doesn't matter since the artificial criteria will always hit anyway

    # -------------- Outer Loop --------------
    while k < max_iter:
        t = 0 # Initialize inner iteration counter

        # Initialize/Reset sums for averaging
        x_eta_total = torch.zeros_like(x)
        y_eta_total = torch.zeros_like(y)
        eta_total = 0

        # Initialize/Reset Previous restart point for primal weighting
        x_last_restart = x.clone()
        y_last_restart = y.clone()

        # --------- Inner Loop ---------
        while k < max_iter:
            k += 1
            x_previous = x.clone() # For checking necessary criteria
            y_previous = y.clone()

            if adaptive:
                # Adaptive step of pdhg
                x, y, eta, eta_hat, j = adaptive_one_step_pdhg(x, y, c, q, K, l, u, m_ineq, eta, omega, theta, k, j)
            else:
                # Fixed step of pdhg
                x, y, eta, eta_hat = fixed_one_step_pdhg(x, y, c, q, K, l, u, m_ineq, eta, omega, theta)
                j += 1

            # Increase iteration counters
            t += 1

            # Update totals
            x_eta_total += eta * x
            y_eta_total += eta * y
            eta_total += eta

            # Update eta
            eta = eta_hat

            # Infeasibility detection
            lam = project_lambda_box(c - K.T @ y, is_neg_inf, is_pos_inf)
            if k > 1:  # Need at least two points to compute differences
                dx = x - x_prev_iter
                dy = y - y_prev_iter
                dlam = lam - lam_prev

                # Dual infeasibility (primal unbounded) detection
                dlam_plus = (-dlam).clamp(min=0)
                dlam_minus = dlam.clamp(min=0)

                # Check dual infeasibility (primal unbounded)
                if m_eq == 0 or (A_mat @ dx).norm() < tol_eq:
                    Gdx = G_mat @ dx if m_ineq > 0 else torch.zeros((1, 1), device=device)
                    if (m_ineq == 0 or torch.all(Gdx >= -tol_feas)) and (c.T @ dx < tol_obj):
                        bounds_ok = True
                        for i in range(x.shape[0]):
                            dx_i = dx[i].item()
                            c_i = c[i].item()
                            l_i = l[i].item()
                            u_i = u[i].item()
                            if not (
                                (not torch.isinf(l[i]) and not torch.isinf(u[i]) and abs(dx_i) <= tol_feas) or
                                (u_i == float('inf') and c_i >= 0 and dx_i >= -tol_feas) or
                                (l_i == -float('inf') and c_i <= 0 and dx_i <= tol_feas)
                            ):
                                bounds_ok = False
                                break
                        if bounds_ok:
                            certificate_flag = "DUAL_INFEASIBLE"
                            certificate_iter = k
                            if verbose:
                                print(f"[PDLP] Dual infeasibility detected at iter {k}")
                            break

                # Primal infeasibility (dual unbounded) detection
                dy_in = dy[:m_ineq] if m_ineq > 0 else torch.zeros((0, 1), device=device)
                dy_eq = dy[m_ineq:] if m_eq > 0 else torch.zeros((0, 1), device=device)
                dual_res = torch.zeros_like(x)
                if m_ineq > 0: dual_res += G_mat.T @ dy_in
                if m_eq > 0: dual_res += A_mat.T @ dy_eq
                dual_res -= dlam

                if dual_res.norm() < tol_dual_eq and (m_ineq == 0 or torch.all(dy_in >= -tol_feas)):
                    dual_combo = 0.0
                    if m_ineq > 0: dual_combo += (h.T @ dy_in).item()
                    if m_eq > 0: dual_combo += (b.T @ dy_eq).item()

                    finite_l = (~torch.isinf(l).view(-1)) & (l.view(-1) != 0)
                    finite_u = (~torch.isinf(u).view(-1)) & (u.view(-1) != 0)

                    if finite_l.any():
                        dual_combo -= (l[finite_l].view(1, -1) @ dlam_minus[finite_l].view(-1, 1)).item()
                    if finite_u.any():
                        dual_combo -= (u[finite_u].view(1, -1) @ dlam_plus[finite_u].view(-1, 1)).item()

                    if dual_combo > -tol_cert:
                        certificate_flag = "PRIMAL_INFEASIBLE"
                        certificate_iter = k
                        if verbose:
                            print(f"[PDLP] Primal infeasibility detected at iter {k}")
                        break

            # Update previous iterates for next infeasibility check
            x_prev_iter.copy_(x)
            y_prev_iter.copy_(y)
            lam_prev.copy_(lam)

            # Check Restart Criteria Every restart_period iterations
            if t % restart_period == 0:
                # Compute averages
                x_avg = x_eta_total / eta_total
                y_avg = y_eta_total / eta_total

                # Compute KKT errors
                KKT_current = KKT_error(x, y, c, q, K, m_ineq, omega, is_neg_inf, is_pos_inf, l_dual, u_dual, device)
                KKT_average = KKT_error(x_avg, y_avg, c, q, K, m_ineq, omega, is_neg_inf, is_pos_inf, l_dual, u_dual, device)
                KKT_min = min(KKT_current, KKT_average)
                KKT_previous = KKT_error(x_previous, y_previous, c, q, K, m_ineq, omega, is_neg_inf, is_pos_inf, l_dual, u_dual, device)

                # Add three kkt passes
                j += 3

                # Check Restart Criteria and update with Restart Candidate
                if KKT_min <= beta[0] * KKT_first: # Sufficient Criteria
                    if verbose:
                        print(f"Sufficient restart at iteration {t} using the",
                              "Average iterate." if KKT_current >= KKT_average else "Current iterate.")
                    (x, y) = (x_avg, y_avg) if KKT_current >= KKT_average else (x, y)
                    break
                elif KKT_min <= beta[1] * KKT_first and KKT_min > KKT_previous: # Necessary Criteria
                    if verbose:
                        print(f"Necessary restart at iteration {t} using the",
                              "Average iterate." if KKT_current >= KKT_average else "Current iterate.")
                    (x, y) = (x_avg, y_avg) if KKT_current >= KKT_average else (x, y)
                    break
                elif t >= beta[2] * k: # Artificial Criteria
                    if verbose:
                        print(f"Artificial restart at iteration {t} using the",
                              "Average iterate." if KKT_current >= KKT_average else "Current iterate.")
                    (x, y) = (x_avg, y_avg) if KKT_current >= KKT_average else (x, y)
                    break

        # ------------- End Inner Loop ------------

        # Handle infeasibility certificate
        if certificate_flag:
            prim_obj = c.T @ x
            if verbose:
                print(f"Terminating due to {certificate_flag} at iteration {k}")
            return x, prim_obj.cpu().item(), k, n, j

        n += 1 # Increase restart loop counter

        if primal_update: # Primal weight update
            omega = primal_weight_update(x_last_restart, x, y_last_restart, y, omega, 0.5)

        KKT_first = KKT_error(x, y, c, q, K, m_ineq, omega, is_neg_inf, is_pos_inf, l_dual, u_dual, device)
        j += 1 # Add one kkt pass

        # Compute primal and dual residuals, and duality gap
        if precondition:
            D_col, D_row, K_unscaled, c_unscaled, q_unscaled, l_unscaled, u_unscaled = data_precond
            l_unscaled[is_neg_inf] = 0
            u_unscaled[is_pos_inf] = 0
            primal_residual, dual_residual, duality_gap, prim_obj, adjusted_dual = compute_residuals_and_duality_gap(
                D_col * x, D_row * y, c_unscaled, q_unscaled, K_unscaled, m_ineq,
                is_neg_inf, is_pos_inf, l_unscaled, u_unscaled
            )
        else:
            primal_residual, dual_residual, duality_gap, prim_obj, adjusted_dual = compute_residuals_and_duality_gap(
                x, y, c, q, K, m_ineq, is_neg_inf, is_pos_inf, l_dual, u_dual
            )
        j += 1 # Add one kkt pass

        if verbose:
            rel_gap = duality_gap.item() / (1 + abs(prim_obj.item()) + abs(adjusted_dual.item()))
            rel_primal = primal_residual.item() / (1 + q_norm)
            rel_dual = dual_residual.item() / (1 + c_norm)

            print(f"[{k}] Primal Obj: {prim_obj.item():.4f}, Adjusted Dual Obj: {adjusted_dual.item():.4f}, "
                  f"Gap: {rel_gap:.2e}, Prim Res: {rel_primal:.2e}, Dual Res: {rel_dual:.2e}")
            print("")

        # Termination conditions
        if check_termination(primal_residual, dual_residual, duality_gap, prim_obj, adjusted_dual, q_norm, c_norm, tol):
            if verbose:
                print(f"Converged at iteration {k} after {n} restart loops")
            break

    # ------------------- End Outer Loop ------------------------

    return x, prim_obj.cpu().item(), k, n, j
     

In [18]:
import torch

def fixed_one_step_pdhg(x, y, c, q, K, l, u, m_ineq, eta, omega, theta):
    """
    Perform one step of the Primal-Dual Hybrid Gradient (PDHG) algorithm.
    Args:
        x (torch.Tensor): Current primal variable.
        y (torch.Tensor): Current dual variable.
        c (torch.Tensor): Coefficients for the primal objective.
        q (torch.Tensor): Right-hand side vector for the constraints.
        K (torch.Tensor): Constraint matrix.
        l (torch.Tensor): Lower bounds for the primal variable.
        u (torch.Tensor): Upper bounds for the primal variable.
        m_ineq (int): Number of inequality constraints.
        eta (float): Step size for the primal update.
        omega (float): Scaling factor for the dual update.
        theta (float): Extrapolation parameter.
    Returns:
        x (torch.Tensor): Updated primal variable.
        y (torch.Tensor): Updated dual variable.
    """
    x_old = x.clone()

    # Compute gradient and primal update
    Kt_y = K.T @ y
    grad = c - Kt_y
    x = torch.clamp(x - eta / omega * grad, min=l, max=u)

    # Extrapolate
    x_bar = x + theta * (x - x_old)

    # Dual update
    K_xbar = K @ x_bar
    y += eta * omega * (q - K_xbar)

    # Project dual:
    if m_ineq > 0:
        y[:m_ineq] = torch.clamp(y[:m_ineq], min=0.0)

    return x, y, eta, eta


def adaptive_one_step_pdhg(x, y, c, q, K, l, u, m_ineq, eta, omega, theta, k, j):
    """
    Perform one step of the Primal-Dual Hybrid Gradient (PDHG) algorithm with adaptive stepsize.
    Args:
        x (torch.Tensor): Current primal variable.
        y (torch.Tensor): Current dual variable.
        c (torch.Tensor): Coefficients for the primal objective.
        q (torch.Tensor): Right-hand side vector for the constraints.
        K (torch.Tensor): Constraint matrix.
        l (torch.Tensor): Lower bounds for the primal variable.
        u (torch.Tensor): Upper bounds for the primal variable.
        m_ineq (int): Number of inequality constraints.
        eta (float): Step size for the primal update.
        omega (float): Scaling factor for the dual update.
        theta (float): Extrapolation parameter.
        k (int): Current iteration number.
        j (int): Current KKT pass number.
    Returns:
        x (torch.Tensor): Updated primal variable.
        y (torch.Tensor): Updated dual variable.
    """
    x_old = x.clone()
    y_old = y.clone()

    # Primal update
    Kt_y = K.T @ y_old
    grad = c - Kt_y

    for i in range(200):

        # --- CURRENT STEPSIZE ---
        tau = eta / omega
        sigma = eta * omega

        x = torch.clamp(x_old - tau * grad, min=l, max=u)

        # Extrapolate
        diff_x = x - x_old
        x_bar = x + theta * diff_x

        # Dual update
        K_xbar = K @ x_bar
        y = y_old + sigma * (q - K_xbar)

        # Project duals:
        if m_ineq > 0:
            y[:m_ineq] = torch.clamp(y[:m_ineq], min=0.0)

        diff_y = y - y_old

        j += 1

        # Calculate the denominator for the eta_bar update
        denominator = 2 * (diff_y.T @ K @ diff_x)

        # --- CALCULATE NEW STEP SIZES ---
        if denominator != 0:
            numerator = omega * (torch.linalg.norm(diff_x)**2) + (torch.linalg.norm(diff_y)**2) / omega
            eta_bar = numerator / abs(denominator)
            eta_prime_term1 = (1 - (k + 1)**(-0.3)) * eta_bar
        else:
            eta_bar = torch.tensor(float('inf'))
            eta_prime_term1 = torch.tensor(float('inf'))

        eta_prime_term2 = (1 + (k + 1)**(-0.6)) * eta
        eta_prime = torch.min(eta_prime_term1, eta_prime_term2)

        if eta <= eta_bar:
            return x, y, eta.squeeze(), eta_prime.squeeze(), j

        eta = eta_prime

        return x, y, eta.squeeze(), eta.squeeze(), j

In [19]:
import torch
import numpy as np
from collections import defaultdict
import time

class Timer:
  """
  Timer class to measure execution time of code blocks.
  Usage:

    with Timer("Label"):
        # Code block to be timed

  Output:
    Label: <time in seconds> seconds
  """
  def __init__(self, label="Elapsed time"):
      self.label = label

  def __enter__(self):
      self.start = time.perf_counter()
      return self

  def __exit__(self, *args):
      self.end = time.perf_counter()
      self.elapsed = self.end - self.start
      print(f"{self.label}: {self.elapsed:.6f} seconds")

def sparse_vs_dense(A, device='cpu', kkt_passes=10):
    """
    Benchmarks matrix-vector multiplication using dense and sparse formats.

    Parameters:
        A (torch.Tensor): 2D matrix (dense tensor)
        device (str): 'cpu' or 'cuda'
        kkt_passes (even int): Number of repetitions for matrix multiplication

    Returns:
        tensor A as either sparse or dense, which ever is faster
    """
    assert A.dim() == 2, "Input must be a 2D matrix"
    m, n = A.shape
    A = A.to(device)

     # Precompute random vectors for fair timing
    vecs_n = [torch.randn(n, 1, device=device) for _ in range(kkt_passes // 2)]
    vecs_m = [torch.randn(m, 1, device=device) for _ in range(kkt_passes // 2)]

    # Dense timing
    A_transpose = A.t()
    if device == 'cuda':
        torch.cuda.synchronize()
    start = time.time()
    for vec_n, vec_m in zip(vecs_n, vecs_m):
        _ = A @ vec_n
        _ = A_transpose @ vec_m
    if device == 'cuda':
        torch.cuda.synchronize()
    dense_time = time.time() - start

    # Sparse timing
    A_sparse = A.to_sparse()
    A_sparse_transpose = A_sparse.t()
    if device == 'cuda':
        torch.cuda.synchronize()
    start = time.time()
    for vec_n, vec_m in zip(vecs_n, vecs_m):
        _ = torch.sparse.mm(A_sparse, vec_n)
        _ = torch.sparse.mm(A_sparse_transpose, vec_m)
    if device == 'cuda':
        torch.cuda.synchronize()
    sparse_time = time.time() - start

    return A_sparse if sparse_time < dense_time else A

def mps_to_standard_form(mps_file, device='cpu', support_sparse=False, verbose=False):
    """
    Parses an MPS file and returns the standard form LP components as PyTorch tensors:
        minimize     cᵀx
        subject to   G x ≥ h
                     A x = b
                     l ≤ x ≤ u

    Returns: c, G, h, A, b, l, u
    """


    #Read MPS file
    with open(mps_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip() and not line.startswith('*')]

    section = None
    row_types = {}
    row_indices = {}
    col_data = defaultdict(list)
    rhs_data = {}
    range_data = {}
    bound_data = defaultdict(dict)

    row_counter = 0
    var_names = []
    seen_vars = set()
    obj_row_name = None

    for line in lines:
        if line == 'NAME' or line == 'ENDATA':
            continue
        elif line == 'ROWS':
            section = 'ROWS'
            continue
        elif line == 'COLUMNS':
            section = 'COLUMNS'
            continue
        elif line == 'RHS':
            section = 'RHS'
            continue
        elif line == 'RANGES':
            section = 'RANGES'
            continue
        elif line == 'BOUNDS':
            section = 'BOUNDS'
            continue

        tokens = line.split()
        if section == 'ROWS':
            sense, row_name = tokens
            row_types[row_name] = sense
            row_indices[row_name] = row_counter
            if sense == 'N':
                obj_row_name = row_name
            row_counter += 1

        elif section == 'COLUMNS':
            var_name = tokens[0]
            if var_name not in seen_vars:
                var_names.append(var_name)
                seen_vars.add(var_name)
            for i in range(1, len(tokens), 2):
                row, val = tokens[i], float(tokens[i + 1])
                col_data[var_name].append((row, val))

        elif section == 'RHS':
            for i in range(1, len(tokens), 2):
                row, val = tokens[i], float(tokens[i + 1])
                rhs_data[row] = val

        elif section == 'RANGES':
            for i in range(1, len(tokens), 2):
                row, val = tokens[i], float(tokens[i + 1])
                range_data[row] = val

        elif section == 'BOUNDS':
            bound_type, _, var_name = tokens[:3]
            val = float(tokens[3]) if len(tokens) > 3 else None
            if bound_type == 'LO':
                bound_data[var_name]['lo'] = val
            elif bound_type == 'UP':
                bound_data[var_name]['up'] = val
            elif bound_type == 'FX':
                bound_data[var_name]['lo'] = val
                bound_data[var_name]['up'] = val
            elif bound_type == 'FR':
                bound_data[var_name]['lo'] = 0.0
                bound_data[var_name]['up'] = float('inf')

    # Final variable ordering and index mapping
    var_index = {v: i for i, v in enumerate(var_names)}
    num_vars = len(var_names)

    # Build objective vector c
    c = np.zeros(num_vars)
    for var, entries in col_data.items():
        col_idx = var_index[var]
        for row_name, val in entries:
            if row_name == obj_row_name:
                c[col_idx] = val

    # Build row vectors from col_data
    row_vectors = {row: np.zeros(num_vars) for row in row_types}
    for var, entries in col_data.items():
        col_idx = var_index[var]
        for row_name, val in entries:
            row_vectors[row_name][col_idx] = val

     # Build A (equality) and G (inequality)
    A_rows, b_eq = [], []
    G_rows, h_ineq = [], []

    for row_name, sense in row_types.items():
        if row_name == obj_row_name:
            continue

        row_vec = row_vectors[row_name]
        rhs_val = rhs_data.get(row_name, 0.0)
        range_val = range_data.get(row_name, None)

        if range_val is not None:
            if sense == 'G':
                lb = rhs_val
                ub = rhs_val + abs(range_val)
            elif sense == 'L':
                ub = rhs_val
                lb = rhs_val - abs(range_val)
            elif sense == 'E':
                if range_val > 0:
                    lb = rhs_val
                    ub = rhs_val + range_val
                else:
                    ub = rhs_val
                    lb = rhs_val + range_val
            else:
                raise ValueError(f"Unsupported ranged sense: {sense}")

            G_rows.append(row_vec)
            h_ineq.append(lb)
            G_rows.append(-row_vec)
            h_ineq.append(-ub)

        else:
            if sense == 'E':
                A_rows.append(row_vec)
                b_eq.append(rhs_val)
            elif sense == 'G':
                G_rows.append(row_vec)
                h_ineq.append(rhs_val)
            elif sense == 'L':
                G_rows.append(-row_vec)
                h_ineq.append(-rhs_val)

    # Bounds
    l = []
    u = []
    for var in var_names:
        lo = bound_data[var].get('lo', 0)
        up = bound_data[var].get('up', float('inf'))
        l.append(lo)
        u.append(up)

    # Convert all to torch
    A_tensor = torch.tensor(np.array(A_rows), dtype=torch.float32, device=device)
    b_tensor = torch.tensor(np.array(b_eq), dtype=torch.float32, device=device).view(-1, 1)
    G_tensor = torch.tensor(np.array(G_rows), dtype=torch.float32, device=device)
    h_tensor = torch.tensor(np.array(h_ineq), dtype=torch.float32, device=device).view(-1, 1)
    c_tensor = torch.tensor(c, dtype=torch.float32, device=device).view(-1, 1)
    l_tensor = torch.tensor(l, dtype=torch.float32, device=device).view(-1, 1)
    u_tensor = torch.tensor(u, dtype=torch.float32, device=device).view(-1, 1)

    m_ineq = G_tensor.shape[0] if G_tensor.numel() > 0 else 0

    # Combine original constraints into K and q
    combined_matrix_list = []
    rhs = []
    if m_ineq > 0:
        combined_matrix_list.append(G_tensor)
        rhs.append(h_tensor)
    if A_tensor.numel() > 0:
        combined_matrix_list.append(A_tensor)
        rhs.append(b_tensor)

    K_tensor = torch.vstack(combined_matrix_list)
    q_tensor = torch.vstack(rhs)

    if support_sparse:
        # Check if sparse operations are faster
        K_tensor = sparse_vs_dense(K_tensor, device=device, kkt_passes=10)
        if verbose:
            print("Using Sparse operations") if K_tensor.is_sparse else print("Using Dense operations")

    return c_tensor, K_tensor, q_tensor, m_ineq, l_tensor, u_tensor


